In [ ]:
from gensim.models import fasttext as ft
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
import operator
import re

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
# https://fasttext.cc/docs/en/crawl-vectors.html
model=ft.load_facebook_model(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))

In [ ]:
window_size=3
stemmer = nltk.wordnet.WordNetLemmatizer()

In [ ]:
def rank(text):
    text=text.lower()
    stops = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    filtered_words = [stemmer.lemmatize(word) for word in tokens if word not in stops]
    scores=dict()
    counts=dict()
    contextSizes=dict()
    for i in range(len(filtered_words)):
        counts.setdefault(filtered_words[i],0)
        counts[filtered_words[i]]+=1
        leftBound=max(0,i-window_size)
        rightBound=min(len(filtered_words)-1,i+window_size)
        contextSize=rightBound-leftBound+1
        contextSizes.setdefault(filtered_words[i],contextSize)
        contextSizes[filtered_words[i]]=max(contextSizes[filtered_words[i]],contextSize)
        for j in range(i+1,rightBound+1):
            similarity_score=model.wv.similarity(filtered_words[i],filtered_words[j])
            similarity_score=(similarity_score+1)/2.0
            scores.setdefault(filtered_words[i],similarity_score)
            scores.setdefault(filtered_words[j],similarity_score)
            scores[filtered_words[i]]=min(scores[filtered_words[i]],similarity_score)
            scores[filtered_words[j]]=min(scores[filtered_words[j]],similarity_score)
    wordScores=list(map(list, scores.items()))
    for i in range(len(wordScores)):
        wordScores[i][1]=wordScores[i][1]/(counts[wordScores[i][0]]*contextSizes[wordScores[i][0]])
    wordScores=sorted(wordScores, key = lambda x: x[1])#,reverse=True)
    filteredWordScores=[]
    for word in wordScores:
        filtered=re.sub('[^a-zA-Z]', '', word[0])
        if(len(filtered)>2):
            filteredWordScores.append([filtered,word[1]])
    return filteredWordScores

In [120]:
rank("fastText is a library for learning of word embeddings and text classification created by Facebook's AI Research lab. The model allows to create an unsupervised learning or supervised learning algorithm for obtaining vector representations for words. Facebook makes available pretrained models for 294 languages. fastText uses a neural network for word embedding.")[:5]

[['learning', 0.023836222380244482],
 ['word', 0.02452107887005522],
 ['fasttext', 0.03222315385937691],
 ['facebook', 0.03594347079550581],
 ['model', 0.03712539082126958]]